In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D

#  25 x  25
model = Sequential() 
model.add(Conv2D(32, (3, 3), activation='relu', padding="valid", input_shape=(500, 500, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
#  12 x  12
model.add(Conv2D(64, (3, 3), activation='relu', padding="valid"))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', padding="valid"))
model.add(MaxPooling2D(pool_size=(2, 2)))
#   6 x   6
model.add(Conv2D(6, (4, 4), activation='relu', padding="valid"))
model.add(GlobalMaxPooling2D())

model.add(Dense(7))

opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 498, 498, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 249, 249, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 247, 247, 64)      18496     
_________________________________________________________________
batch_normalization (BatchNo (None, 247, 247, 64)      256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 245, 245, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 122, 122, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 119, 119, 6)       6

In [ ]:
%pwd

'/content'

In [ ]:
!unzip -q train_data_prepared.zip

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(validation_split=0.1)
train_generator = datagen.flow_from_directory('train_data/', target_size=(500, 500), class_mode='categorical', subset='training')
valid_generator = datagen.flow_from_directory('train_data/', target_size=(500, 500), class_mode='categorical', subset='validation')

Found 24343 images belonging to 7 classes.
Found 2701 images belonging to 7 classes.


In [ ]:
model.fit(train_generator,
          validation_data=valid_generator,
          steps_per_epoch = train_generator.n//train_generator.batch_size,
          validation_steps = valid_generator.n//valid_generator.batch_size,
          epochs=5)

Epoch 1/5
760/760 [==============================] - 277s 319ms/step - loss: 7.6427 - accuracy: 0.1684 - val_loss: 7.6693 - val_accuracy: 0.1696
Epoch 2/5
760/760 [==============================] - 237s 311ms/step - loss: 7.6722 - accuracy: 0.1698 - val_loss: 7.6813 - val_accuracy: 0.1708
Epoch 3/5
760/760 [==============================] - 234s 308ms/step - loss: 7.6762 - accuracy: 0.1697 - val_loss: 7.6753 - val_accuracy: 0.1700
Epoch 4/5
760/760 [==============================] - 231s 304ms/step - loss: 7.6788 - accuracy: 0.1697 - val_loss: 7.6753 - val_accuracy: 0.1685
Epoch 5/5
760/760 [==============================] - 227s 298ms/step - loss: 7.6768 - accuracy: 0.1696 - val_loss: 7.6633 - val_accuracy: 0.1708


In [ ]:
score = model.evaluate(valid_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

85/85 [==============================] - 20s 239ms/step - loss: 7.6741 - accuracy: 0.1699
Test loss: 7.674147605895996
Test accuracy: 0.16993705928325653


In [ ]:
model.save('model')

In [ ]:
!zip -r model.zip model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D

CLASS_COUNT = 6

#  25 x  25
model = Sequential() 
model.add(Conv2D(32, (3, 3), activation='relu', padding="valid", input_shape=(500, 500, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
#  12 x  12
model.add(Conv2D(64, (3, 3), activation='relu', padding="valid"))
model.add(MaxPooling2D(pool_size=(2, 2)))
#   6 x   6
model.add(Conv2D(CLASS_COUNT, (4, 4), activation='relu', padding="valid"))
model.add(GlobalMaxPooling2D())

model.add(Dense(CLASS_COUNT + 1))

opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
from random import choices, seed

seed(42)

for i in range(3):
    print('Iteration:', i)
    x, y = [], []
    for cls in range(7):
        encode = [0, 0, 0, 0, 0, 0]
        if cls:
            encode[cls - 1] = 1

        fn = choices(train_filename_by_class[cls], k=5)
        for path in fn:
            #y.append(encode)
            #x.append(cv2.cvtColor(prepare_image(cv2.imread(path)), cv2.COLOR_BGR2RGB))
            model.fit(np.array([cv2.cvtColor(prepare_image(cv2.imread(path)), cv2.COLOR_BGR2RGB)]) / 255, 
                      np.array([encode]))

    #print(np.array(x).shape)
    #print(np.array(y).shape)
    #model.fit(tf.convert_to_tensor(np.array(x)), tf.convert_to_tensor(np.array(y)))

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.utils  import plot_model

In [ ]:
model = load_model("model")

In [ ]:
plot_model(model)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [ ]:
seed = 21
class_num = 4
inshape = (50, 50, 1)

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=inshape, padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dropout(0.2))

model.add(Dense(256, kernel_constraint=max_norm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(128, kernel_constraint=max_norm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(class_num))
model.add(Activation('softmax'))

epochs = 25
optimizer = 'Adam'

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_115 (Conv2D)          (None, 50, 50, 32)        320       
_________________________________________________________________
activation_131 (Activation)  (None, 50, 50, 32)        0         
_________________________________________________________________
dropout_54 (Dropout)         (None, 50, 50, 32)        0         
_________________________________________________________________
batch_normalization_129 (Bat (None, 50, 50, 32)        128       
_________________________________________________________________
conv2d_116 (Conv2D)          (None, 50, 50, 64)        18496     
_________________________________________________________________
activation_132 (Activation)  (None, 50, 50, 64)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 25, 25, 64)       

In [ ]:
model.fit(train_generator,
          validation_data = train_generator,
          steps_per_epoch = train_generator.n//train_generator.batch_size,
          validation_steps = valid_generator.n//valid_generator.batch_size,
          epochs=epochs)

Epoch 1/25
779/779 [==============================] - 121s 153ms/step - loss: 1.4877 - accuracy: 0.3629 - val_loss: 1.2657 - val_accuracy: 0.4601
Epoch 2/25
779/779 [==============================] - 121s 156ms/step - loss: 1.3514 - accuracy: 0.3876 - val_loss: 1.2010 - val_accuracy: 0.4433
Epoch 3/25
779/779 [==============================] - 128s 165ms/step - loss: 1.2602 - accuracy: 0.4310 - val_loss: 1.1487 - val_accuracy: 0.5000
Epoch 4/25
779/779 [==============================] - 122s 157ms/step - loss: 1.2120 - accuracy: 0.4669 - val_loss: 1.1881 - val_accuracy: 0.5000
Epoch 5/25
779/779 [==============================] - 120s 155ms/step - loss: 1.2403 - accuracy: 0.4557 - val_loss: 1.0823 - val_accuracy: 0.5554
Epoch 6/25
779/779 [==============================] - 122s 157ms/step - loss: 1.2084 - accuracy: 0.4788 - val_loss: 0.9631 - val_accuracy: 0.6070
Epoch 7/25
779/779 [==============================] - 122s 157ms/step - loss: 1.1870 - accuracy: 0.4801 - val_loss: 0.9561 -

In [ ]:
score = model.evaluate(valid_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

195/195 [==============================] - 5s 25ms/step - loss: 1.0034 - accuracy: 0.5546
Test loss: 1.003422737121582
Test accuracy: 0.5545571446418762


In [ ]:
model.save('model5')

INFO:tensorflow:Assets written to: model5\assets


In [ ]:
next(train_generator)

(array([[[[0.        ],
          [0.        ],
          [0.        ],
          ...,
          [0.10802189],
          [0.        ],
          [0.        ]],
 
         [[0.        ],
          [0.        ],
          [0.        ],
          ...,
          [0.02156932],
          [0.        ],
          [0.        ]],
 
         [[0.        ],
          [0.        ],
          [0.        ],
          ...,
          [0.        ],
          [0.        ],
          [0.11078141]],
 
         ...,
 
         [[0.        ],
          [0.        ],
          [0.        ],
          ...,
          [0.7879385 ],
          [0.45598555],
          [0.12403262]],
 
         [[0.        ],
          [0.        ],
          [0.        ],
          ...,
          [0.25329763],
          [0.5852506 ],
          [0.91720355]],
 
         [[0.        ],
          [0.        ],
          [0.        ],
          ...,
          [0.        ],
          [0.        ],
          [0.        ]]],
 
 
        [

In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from tensorflow.keras.layers import Flatten, MaxPooling2D, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization
from tensorflow.keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU
from tensorflow.keras.layers import AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.constraints import max_norm
from IPython.display import SVG

from time import time
import numpy as np
import os

In [ ]:
src_path_train = os.path.join('data', 'train')
src_path_test  = os.path.join('data', 'test')

train_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest",
        validation_split=0.20)

test_datagen = ImageDataGenerator(rescale=1 / 255.0)

In [ ]:
batch_size = 6
train_generator = train_datagen.flow_from_directory(
    directory=src_path_train,
    target_size=(50, 50),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    subset='training',
    shuffle=True,
    seed=42
)
valid_generator = train_datagen.flow_from_directory(
    directory=src_path_train,
    target_size=(50, 50),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical",
    subset='validation',
    shuffle=True,
    seed=42
)
test_generator = test_datagen.flow_from_directory(
    directory=src_path_test,
    target_size=(50, 50),
    color_mode="grayscale",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

Found 3730 images belonging to 5 classes.
Found 932 images belonging to 5 classes.
Found 157 images belonging to 1 classes.


In [ ]:
def densenet(img_shape, n_classes, f=32):
    repetitions = 6, 12, 24, 16
    
    def bn_rl_conv(x, f, k=1, s=1, p='same'):
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(f, k, strides=s, padding=p)(x)
        return x
  
    def dense_block(tensor, r):
        for _ in range(r):
            x = bn_rl_conv(tensor, 4*f)
            x = bn_rl_conv(x, f, 3)
            tensor = Concatenate()([tensor, x])
        return tensor
  
  
    def transition_block(x):
        x = bn_rl_conv(x, K.int_shape(x)[-1] // 2)
        x = AvgPool2D(2, strides=2, padding='same')(x)
        return x
    
    
    input = Input(img_shape)
    
    x = Conv2D(64, 7, strides=2, padding='same')(input)
    x = MaxPool2D(3, strides=2, padding='same')(x)
    
    for r in repetitions:
        d = dense_block(x, r)
        x = transition_block(d)
    
    x = GlobalAvgPool2D()(d)
    output = Dense(n_classes, activation='softmax')(x)
    
    model = Model(input, output)
    return model

In [ ]:
model = densenet((50, 50, 1), 5, 16)

In [ ]:
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, 50, 50, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_361 (Conv2D)             (None, 25, 25, 64)   3200        input_16[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_12 (MaxPooling2D) (None, 13, 13, 64)   0           conv2d_361[0][0]                 
__________________________________________________________________________________________________
batch_normalization_375 (BatchN (None, 13, 13, 64)   256         max_pooling2d_12[0][0]           
____________________________________________________________________________________________

In [ ]:
model.fit(train_generator,
          validation_data = valid_generator,
          steps_per_epoch = train_generator.n//train_generator.batch_size,
          validation_steps = valid_generator.n//valid_generator.batch_size,
          epochs=50)

Epoch 1/50
621/621 [==============================] - 256s 288ms/step - loss: 1.3462 - accuracy: 0.5255 - val_loss: 2.4370 - val_accuracy: 0.5194
Epoch 2/50
621/621 [==============================] - 144s 232ms/step - loss: 0.7565 - accuracy: 0.7301 - val_loss: 3.0308 - val_accuracy: 0.5344
Epoch 3/50
621/621 [==============================] - 146s 235ms/step - loss: 0.6019 - accuracy: 0.7884 - val_loss: 1.0237 - val_accuracy: 0.7226
Epoch 4/50
621/621 [==============================] - 148s 238ms/step - loss: 0.4991 - accuracy: 0.8265 - val_loss: 1.8360 - val_accuracy: 0.6699
Epoch 5/50
621/621 [==============================] - 147s 237ms/step - loss: 0.4412 - accuracy: 0.8526 - val_loss: 1.7156 - val_accuracy: 0.6505
Epoch 6/50
621/621 [==============================] - 142s 228ms/step - loss: 0.3946 - accuracy: 0.8676 - val_loss: 0.9651 - val_accuracy: 0.7645
Epoch 7/50
621/621 [==============================] - 142s 228ms/step - loss: 0.3405 - accuracy: 0.8904 - val_loss: 0.8185 -

In [ ]:
score = model.evaluate(valid_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

156/156 [==============================] - 8s 52ms/step - loss: 0.8476 - accuracy: 0.8573
Test loss: 0.8476055860519409
Test accuracy: 0.8572961091995239


In [ ]:
predict=model.predict(test_generator)
# predict the class label
y_classes = predict.argmax(axis=-1)

In [ ]:
model.output_shape

(None, 5)

In [ ]:
model.save('model')

INFO:tensorflow:Assets written to: model\assets


In [ ]:
def prepare_model():
    model = Sequential()
    model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(50, 50, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(16, activation='relu'))
    model.add(Dense(4, activation='sigmoid'))
    model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])
    return model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Input, Concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

In [ ]:
def conv_factory(x, concat_axis, nb_filter,
                 dropout_rate=None, weight_decay=1E-4):
    """Apply BatchNorm, Relu 3x3Conv2D, optional dropout
    :param x: Input keras network
    :param concat_axis: int -- index of contatenate axis
    :param nb_filter: int -- number of filters
    :param dropout_rate: int -- dropout rate
    :param weight_decay: int -- weight decay factor
    :returns: keras network with b_norm, relu and Conv2D added
    :rtype: keras network
    """

    x = BatchNormalization(axis=concat_axis,
                           gamma_regularizer=l2(weight_decay),
                           beta_regularizer=l2(weight_decay))(x)
    x = Activation('relu')(x)
    x = Conv2D(nb_filter, (3, 3),
               kernel_initializer="he_uniform",
               padding="same",
               use_bias=False,
               kernel_regularizer=l2(weight_decay))(x)
    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    return x


def transition(x, concat_axis, nb_filter,
               dropout_rate=None, weight_decay=1E-4):
    """Apply BatchNorm, Relu 1x1Conv2D, optional dropout and Maxpooling2D
    :param x: keras model
    :param concat_axis: int -- index of contatenate axis
    :param nb_filter: int -- number of filters
    :param dropout_rate: int -- dropout rate
    :param weight_decay: int -- weight decay factor
    :returns: model
    :rtype: keras model, after applying batch_norm, relu-conv, dropout, maxpool
    """

    x = BatchNormalization(axis=concat_axis,
                           gamma_regularizer=l2(weight_decay),
                           beta_regularizer=l2(weight_decay))(x)
    x = Activation('relu')(x)
    x = Conv2D(nb_filter, (1, 1),
               kernel_initializer="he_uniform",
               padding="same",
               use_bias=False,
               kernel_regularizer=l2(weight_decay))(x)
    if dropout_rate:
        x = Dropout(dropout_rate)(x)
    x = AveragePooling2D((2, 2), strides=(2, 2))(x)

    return x


def denseblock(x, concat_axis, nb_layers, nb_filter, growth_rate,
               dropout_rate=None, weight_decay=1E-4):
    """Build a denseblock where the output of each
       conv_factory is fed to subsequent ones
    :param x: keras model
    :param concat_axis: int -- index of contatenate axis
    :param nb_layers: int -- the number of layers of conv_
                      factory to append to the model.
    :param nb_filter: int -- number of filters
    :param dropout_rate: int -- dropout rate
    :param weight_decay: int -- weight decay factor
    :returns: keras model with nb_layers of conv_factory appended
    :rtype: keras model
    """

    list_feat = [x]

    for i in range(nb_layers):
        x = conv_factory(x, concat_axis, growth_rate,
                         dropout_rate, weight_decay)
        list_feat.append(x)
        x = Concatenate(axis=concat_axis)(list_feat)
        nb_filter += growth_rate

    return x, nb_filter


def denseblock_altern(x, concat_axis, nb_layers, nb_filter, growth_rate,
                      dropout_rate=None, weight_decay=1E-4):
    """Build a denseblock where the output of each conv_factory
       is fed to subsequent ones. (Alternative of a above)
    :param x: keras model
    :param concat_axis: int -- index of contatenate axis
    :param nb_layers: int -- the number of layers of conv_
                      factory to append to the model.
    :param nb_filter: int -- number of filters
    :param dropout_rate: int -- dropout rate
    :param weight_decay: int -- weight decay factor
    :returns: keras model with nb_layers of conv_factory appended
    :rtype: keras model
    * The main difference between this implementation and the implementation
    above is that the one above
    """

    for i in range(nb_layers):
        merge_tensor = conv_factory(x, concat_axis, growth_rate,
                                    dropout_rate, weight_decay)
        x = Concatenate(axis=concat_axis)([merge_tensor, x])
        nb_filter += growth_rate

    return x, nb_filter


def DenseNet(nb_classes, img_dim, depth, nb_dense_block, growth_rate,
             nb_filter, dropout_rate=None, weight_decay=1E-4):
    """ Build the DenseNet model
    :param nb_classes: int -- number of classes
    :param img_dim: tuple -- (channels, rows, columns)
    :param depth: int -- how many layers
    :param nb_dense_block: int -- number of dense blocks to add to end
    :param growth_rate: int -- number of filters to add
    :param nb_filter: int -- number of filters
    :param dropout_rate: float -- dropout rate
    :param weight_decay: float -- weight decay
    :returns: keras model with nb_layers of conv_factory appended
    :rtype: keras model
    """
    
    if K.image_data_format() == "channels_first":
        concat_axis = 1
    elif K.image_data_format() == "channels_last":
        concat_axis = -1

    model_input = Input(shape=img_dim)

    assert (depth - 4) % 3 == 0, "Depth must be 3 N + 4"

    # layers in each dense block
    nb_layers = int((depth - 4) / 3)

    # Initial convolution
    x = Conv2D(nb_filter, (3, 3),
               kernel_initializer="he_uniform",
               padding="same",
               name="initial_conv2D",
               use_bias=False,
               kernel_regularizer=l2(weight_decay))(model_input)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        x, nb_filter = denseblock(x, concat_axis, nb_layers,
                                  nb_filter, growth_rate, 
                                  dropout_rate=dropout_rate,
                                  weight_decay=weight_decay)
        # add transition
        x = transition(x, concat_axis, nb_filter, dropout_rate=dropout_rate,
                       weight_decay=weight_decay)

    # The last denseblock does not have a transition
    x, nb_filter = denseblock(x, concat_axis, nb_layers,
                              nb_filter, growth_rate, 
                              dropout_rate=dropout_rate,
                              weight_decay=weight_decay)

    x = BatchNormalization(axis=concat_axis,
                           gamma_regularizer=l2(weight_decay),
                           beta_regularizer=l2(weight_decay))(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D(data_format=K.image_data_format())(x)
    x = Dense(nb_classes,
              activation='softmax',
              kernel_regularizer=l2(weight_decay),
              bias_regularizer=l2(weight_decay))(x)

    densenet = Model(inputs=[model_input], outputs=[x], name="DenseNet")

    return densenet

In [ ]:
%cd ..

C:\Users\Дима\Desktop\Диплом\Код


In [ ]:
EPOCHS = 50
BATCH_SIZE = 6
PATH = 'data'

train_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        rotation_range=20,
        zoom_range=0.05,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        horizontal_flip=True,
        fill_mode="nearest",
        validation_split=0.20)

train_generator = train_datagen.flow_from_directory(
    directory=PATH,
    target_size=(50, 50),
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset='training',
    shuffle=True,
    seed=42
)
valid_generator = train_datagen.flow_from_directory(
    directory=PATH,
    target_size=(50, 50),
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset='validation',
    shuffle=True,
    seed=42
)

Found 3931 images belonging to 5 classes.
Found 979 images belonging to 5 classes.


In [ ]:
model = DenseNet(5, (50, 50, 1), 13, 5, 32, 16, dropout_rate=0.2)

In [ ]:
opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=["accuracy"])

In [ ]:
model.summary()

Model: "DenseNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           [(None, 50, 50, 1)]  0                                            
__________________________________________________________________________________________________
initial_conv2D (Conv2D)         (None, 50, 50, 16)   144         input_20[0][0]                   
__________________________________________________________________________________________________
batch_normalization_560 (BatchN (None, 50, 50, 16)   64          initial_conv2D[0][0]             
__________________________________________________________________________________________________
activation_203 (Activation)     (None, 50, 50, 16)   0           batch_normalization_560[0][0]    
___________________________________________________________________________________________

In [ ]:
model.fit(train_generator,
          validation_data = valid_generator,
          steps_per_epoch = train_generator.n//train_generator.batch_size,
          validation_steps = valid_generator.n//valid_generator.batch_size,
          epochs=30)

Epoch 1/30
655/655 [==============================] - 367s 551ms/step - loss: 1.8959 - accuracy: 0.5643 - val_loss: 3.2165 - val_accuracy: 0.4356
Epoch 2/30
655/655 [==============================] - 363s 554ms/step - loss: 1.4802 - accuracy: 0.7136 - val_loss: 2.0217 - val_accuracy: 0.5716
Epoch 3/30
655/655 [==============================] - 371s 566ms/step - loss: 1.3088 - accuracy: 0.7549 - val_loss: 1.9316 - val_accuracy: 0.5706
Epoch 4/30
655/655 [==============================] - 363s 554ms/step - loss: 1.1582 - accuracy: 0.8061 - val_loss: 1.6588 - val_accuracy: 0.6677
Epoch 5/30
655/655 [==============================] - 361s 552ms/step - loss: 1.0636 - accuracy: 0.8143 - val_loss: 1.5133 - val_accuracy: 0.6871
Epoch 6/30
655/655 [==============================] - 361s 552ms/step - loss: 0.9923 - accuracy: 0.8408 - val_loss: 1.6939 - val_accuracy: 0.6360
Epoch 7/30
655/655 [==============================] - 361s 551ms/step - loss: 0.9729 - accuracy: 0.8344 - val_loss: 1.7174 -

In [ ]:
score = model.evaluate(valid_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

164/164 [==============================] - 20s 120ms/step - loss: 0.9176 - accuracy: 0.8202
Test loss: 0.9175734519958496
Test accuracy: 0.8202247023582458


In [ ]:
model.save('modelhard')

INFO:tensorflow:Assets written to: modelhard\assets


In [ ]:
from keras.preprocessing import image
img = image.load_img("187.png", color_mode='grayscale')
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)

In [ ]:
model.predict(img)

array([[0., 0., 0., 1.]], dtype=float32)

In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 50, 50, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_245 (Conv2D)             (None, 25, 25, 64)   3200        input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 13, 13, 64)   0           conv2d_245[0][0]                 
__________________________________________________________________________________________________
batch_normalization_240 (BatchN (None, 13, 13, 64)   256         max_pooling2d_5[0][0]            
____________________________________________________________________________________________

Total params: 1,863,760
Trainable params: 1,821,632
Non-trainable params: 42,128
__________________________________________________________________________________________________


In [ ]:
model = keras.models.load_model("model")

score = model.evaluate(valid_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

195/195 [==============================] - 11s 42ms/step - loss: 0.6824 - accuracy: 0.8665
Test loss: 0.6824151277542114
Test accuracy: 0.8664954900741577


In [ ]:
model = keras.models.load_model("modelplzwork")

score = model.evaluate(valid_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

195/195 [==============================] - 4s 16ms/step - loss: 1.2486 - accuracy: 0.7754
Test loss: 1.2485579252243042
Test accuracy: 0.7753530144691467


In [ ]:
model = keras.models.load_model("modeldense")

score = model.evaluate(valid_generator)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

195/195 [==============================] - 3s 13ms/step - loss: 0.6758 - accuracy: 0.7933
Test loss: 0.6757633686065674
Test accuracy: 0.7933247685432434
